# Find hg19 genes

Matteo Bortoletto, Clelia Corridori, Filippo Costa, Edoardo Spadetto

Group: Ragù

In [1]:
%%bash
awk -v OFS='\t' '{if($3=="transcript") print $1,$4,$5,$10}' hg19.refGene.gtf | \
tr -d '\"' | sed 's/\;//g' | sort | uniq > gene_coords.txt
echo
head gene_coords.txt
echo
wc -l gene_coords.txt


chr10	100007448	100027951	LOXL4
chr10	100143325	100174939	PYROXD2
chr10	100154975	100155064	MIR1287
chr10	100175955	100206692	HPS1
chr10	100175955	100206720	HPS1
chr10	100188903	100206720	HPS1
chr10	100191049	100191117	MIR4685
chr10	100206078	100213562	LOC101927278
chr10	100216834	100995609	HPSE2
chr10	100216834	100995632	HPSE2

46562 gene_coords.txt


In [2]:
%%bash
cut -f4 gene_coords.txt | sort | uniq | wc -l

28265


In [4]:
%%bash
cut -f4 gene_coords.txt | sort | uniq | grep -v "LOC" | grep -v "MIR" | grep -v "LINC" | wc -l

22977


In [6]:
import pandas as pd

gene_coords = pd.read_csv('gene_coords.txt', header=None, sep="\t")
gene_coords.sort_values(by = [0, 1, 2], inplace = True)
gene_coords.to_csv('gene_coords.txt', index = False, header = False, sep = '\t')
gene_coords.head()

gene_coords.shape

(46562, 4)

In [8]:
import os
import subprocess

with open('hg19_gene_regions.fa', 'a') as fp:
    for i in range(gene_coords.shape[0]):
        coord = str(str(gene_coords.iloc[i, 0]) + ':' 
                    + str(gene_coords.iloc[i, 1]) + '-' + str(gene_coords.iloc[i, 2]))
        subprocess.run(['samtools', 'faidx', 'hg19.fa.bgz', str(coord)], stdout = fp)

In [10]:
chr_sizes = pd.read_csv("hg19.fa.bgz.fai", header = None, sep = "\t")
chr_sizes = chr_sizes.drop([2, 3, 4], axis = 1)

chr_sizes.head()

,0,1
0,chr1,249250621
1,chr2,243199373
2,chr3,198022430
3,chr4,191154276
4,chr5,180915260


In [11]:
import numpy as np
chr_list = []
start_list = []
end_list = []
gene_lengths = list(gene_coords.iloc[:, 2] - gene_coords.iloc[:, 1])
a = 0
for i in range(gene_coords.shape[0]):
    chr_df = gene_coords[gene_coords[0].isin([gene_coords.iloc[i,0]])]
    overlap = True
    while overlap == True:
        reg_start = np.random.randint(1, int(chr_sizes[chr_sizes[0] == gene_coords.iloc[i,0]].iloc[:,1]))
        reg_end = reg_start + gene_lengths[i]
        for j in range(chr_df.shape[0]):
            b1 = chr_df.iloc[j,1]
            b2 = chr_df.iloc[j,2]
            if (reg_start > b1 and reg_start < b2) or (reg_end > b1 and reg_end < b2) or \
            (b1 > reg_start and b1 < reg_end) or (b2 > reg_start and b2 < reg_end):
                overlap = True
                break
            else:
                overlap = False
    chr_list.append(gene_coords.iloc[i,0])
    start_list.append(reg_start)
    end_list.append(reg_end)
    a = a + 1
    if a%10000 == 0:
            print('Finished ' + str(a) + ' genes')
notgene_coords = pd.DataFrame({'0': chr_list, '1': start_list, '2': end_list})
notgene_coords.to_csv("notgene_coords.txt", index = False, header = False, sep = "\t")
notgene_coords.head()

Finished 10000 genes
Finished 20000 genes
Finished 30000 genes
Finished 40000 genes


,0,1,2
0,chr1,18787054,18789547
1,chr1,112858070,112860605
2,chr1,96545273,96560281
3,chr1,222592548,222592615
4,chr1,37204445,37204512


In [12]:
!bedtools intersect -a gene_coords.txt -b notgene_coords.txt | wc -l

0


In [13]:
import os
import subprocess
with open('hg19_notgene_regions.fa', 'a') as fp:
    for i in range(notgene_coords.shape[0]):
        coord = str(str(notgene_coords.iloc[i, 0]) + ':' 
                    + str(notgene_coords.iloc[i, 1]) + '-' + str(notgene_coords.iloc[i, 2]))
        subprocess.run(['samtools', 'faidx', 'hg19.fa.bgz', str(coord)], stdout = fp)

In [14]:
!grep -c N hg19_gene_regions.fa

141245


In [15]:
!grep -c N hg19_notgene_regions.fa

10810396


In [16]:
from Bio import SeqIO

gene_file = 'hg19_gene_regions.fa'
notgene_file = 'hg19_notgene_regions.fa'
a = 0
i = 0
with open('hg19_gene_clean.fa', 'a') as gene_out, open('hg19_notgene_clean.fa', 'a') as notgene_out:
    for gene, notgene in zip(SeqIO.parse(gene_file, 'fasta'), SeqIO.parse(notgene_file, 'fasta')):
        upper_gene = gene.seq.upper()
        upper_notgene = notgene.seq.upper()
        a = a + 1
        if a%10000 == 0:
            print('Finished ' + str(a) + ' entries')
        if 'N' not in str(upper_gene) and 'N' not in str(upper_notgene):
            gene.seq = upper_gene
            SeqIO.write(gene, gene_out, 'fasta')
            notgene.seq = upper_notgene
            SeqIO.write(notgene, notgene_out, 'fasta')
            i = i + 1
        else:
            continue
print('We have processed ' + str(a) + ' entries and written ' + str(i) + ' entries to two fasta-files')

Finished 10000 entries
Finished 20000 entries
Finished 30000 entries
Finished 40000 entries
We have processed 46562 entries and written 38284 entries to two fasta-files


In [17]:
!grep -c N hg19_gene_clean.fa

0


In [18]:
!grep -c N hg19_notgene_clean.fa

0
